In [12]:
import pandas as pd
import urllib.parse
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:

df_main = pd.read_csv("properties_data.csv", low_memory=False)
df = df_main[["id","location","Zip","Type","Subtype",
    "Price","Transaction Type","Bedrooms","Living area","Kitchen type",
    "Furnished","How many fireplaces?","Terrace","Terrace surface",
    "Garden","Garden surface","Surface of the plot","Number of frontages",
    "Swimming pool","Building condition","Primary energy consumption"]]
df = df.set_index("id")
df = df.drop(df[df["Type"]=="house group"].index)
df = df.drop(df[df["Type"]=="apartment group"].index)
df = df.rename(columns={
    'location' :'Locality',
    'Transaction Type' : 'Type of sale',
    'Type' :'Type of property',
    'Subtype' : 'Subtype of property',
    'Number of frontages': 'Number of facades',
    'Bedrooms':'Number of rooms',
    'Surface of the plot' :'Surface of the land',
    'Kitchen type' : 'Fully equipped kitchen',
    'How many fireplaces?' : 'Open fire'
})
 
#put the columns in a specific order'
df = df.reindex(columns=['Locality',"Zip", 'Type of property', 'Subtype of property',
                         'Type of sale', 'Price',
                         'Building condition','Building Cond. values',
                         'Number of facades', 'Number of rooms', 
                         'Living area',
                         'Furnished','Fully equipped kitchen','Kitchen values',
                         'Surface of the land',
                         'Primary energy consumption','Energy_classes' ,
                         'Terrace', 'Terrace surface','Garden','Garden surface',
                         'Open fire', 'Swimming pool'])

def clean_and_convert(column):
    column = column.apply(lambda x: re.sub('\D+', '', str(x)))
    column = column.replace('', np.nan)
    return column

df['Living area'] = clean_and_convert(df['Living area'])
df['Terrace surface'] = clean_and_convert(df['Terrace surface'])
df['Garden surface'] = clean_and_convert(df['Garden surface'])
df['Surface of the land'] = clean_and_convert(df['Surface of the land'])
df['Primary energy consumption'] = clean_and_convert(df['Primary energy consumption'])
df['Locality'] = df['Locality'].apply(urllib.parse.unquote)

# missing_value_df = df.isnull().mean() * 100
# missing_value_df

conditions = [
    (df['Garden']== "Yes"),
    (df["Garden"].isna()) & (df["Garden surface"].isna()),
    (df["Garden surface"].notna())
    ]
values = [1, 0, 1]
df['Garden'] = np.select(conditions, values)

df.loc[(df["Garden"] == 0 ) & (df["Garden surface"].isna()), 'Garden surface'] = 0
conditions = [
    (df['Terrace']== "Yes"),
    (df["Terrace"].isna()) & (df["Terrace surface"].isna()),
    (df["Terrace surface"].notna())
    ]
values = [1, 0, 1]
df['Terrace'] = np.select(conditions, values)

df.loc[(df["Terrace"] == 0 ) & (df["Terrace surface"].isna()), 'Terrace surface'] = 0
def nan_replacement(column):
    column = column.replace("Yes",1)
    column = column.replace("No",0)
    column = column.replace('', np.nan).fillna(0)
    return column

df['Furnished'] = nan_replacement(df['Furnished'])
df['Swimming pool'] = nan_replacement(df['Swimming pool'])
df['Open fire'] = nan_replacement(df['Open fire'])
df["Price"] = df["Price"].astype(int)
df = df.astype({"Price":"int",
                "Number of rooms":"float",
                "Living area":"float",
                "Terrace surface":"float",
                "Garden surface":"float",
                "Surface of the land":"float",
                "Number of facades":"float",
                "Primary energy consumption":"float"})
# Mapping dictionary for replacing values in the "kitchen" column
kitchen_mapping = {
    np.nan: -1,
    'Not installed': 0,
    'Installed': 1,
    'Semi equipped': 2,
    'Hyper equipped': 3,
    'USA uninstalled' :0,
    'USA installed': 1,
    'USA semi equipped': 2,
    'USA hyper equipped' :3
}
# Replace values in the "Kitchen type" column with corresponding numbers and create a new column called "Kitchen values"
df['Kitchen values'] = df['Fully equipped kitchen'].map(kitchen_mapping).fillna(df['Fully equipped kitchen']).astype(int)
building_cond_mapping = {
    np.nan: -1,
    'To restore': 0,
    'To be done up': 2,
    'Just renovated': 3,
    'To renovate': 1,
    'Good': 3,
    'As new' :4
}

df['Building Cond. values'] = df['Building condition'].map(building_cond_mapping).fillna(df['Building condition']).astype(int)
df = df.drop(df[df["Living area"].isna()].index)
#Missing values fillied with 1
df["Primary energy consumption"] = np.where((df["Primary energy consumption"] != int) & (df["Primary energy consumption"] == ""), 0, df["Primary energy consumption"])
df["Primary energy consumption"] = df["Primary energy consumption"].replace("",np.nan).fillna(-1).astype(int)

#New column with energy classes 
conditions = [
    (df['Primary energy consumption']==-1),
    (df['Primary energy consumption']>=1)&(df['Primary energy consumption']<100),
    (df['Primary energy consumption']>=100)&(df['Primary energy consumption']<200),
    (df['Primary energy consumption']>=200)&(df['Primary energy consumption']<300),
    (df['Primary energy consumption']>=300)&(df['Primary energy consumption']<400),
    (df['Primary energy consumption']>=400)&(df['Primary energy consumption']<500),
    (df['Primary energy consumption']>=500)&(df['Primary energy consumption']<600),
    (df['Primary energy consumption']>=600)
]

values = [-1,7, 6, 5, 4, 3, 2, 1]

df['Energy_classes'] = np.select(conditions, values)
print(f"The DataFrame has {df.shape[0]} rows and {df.shape[1]} columns.")
# print(missing_value_df)
df.to_csv("clean_data_MouradTest.csv")

The DataFrame has 18030 rows and 23 columns.


In [14]:
'''df = pd.read_csv('properties_data.csv')
df = df.drop_duplicates(subset='id')
df = df.drop(df[df["Type"].isin(["house group", "apartment group"])].index)

duplicate_columns = df.columns[df.columns.duplicated()]
df = df.loc[:, ~df.columns.duplicated()]

filtered_columns = ["id","location","Zip","Type","Subtype",
    "Price","Transaction Type","Bedrooms","Living area","Kitchen type",
    "Furnished","How many fireplaces?","Terrace","Terrace surface",
    "Garden","Garden surface","Surface of the plot","Number of frontages",
    "Swimming pool","Building condition","Primary energy consumption"]
df = df[filtered_columns]

df = df.rename(columns={
    'location' :'Locality',
    'Transaction Type' : 'Type of sale',
    'Type' :'Type of property',
    'Subtype' : 'Subtype of property',
    'Number of frontages': 'Number of facades',
    'Bedrooms':'Number of rooms',
    'Surface of the plot' :'Surface of the land',
    'Kitchen type' : 'Fully equipped kitchen',
    'How many fireplaces?' : 'Open fire'
})
 
#put the columns in a specific order'
df = df.reindex(columns=['id', 'Locality',"Zip", 'Type of property', 'Subtype of property',
                         'Type of sale', 'Price',
                         'Building condition','Building Cond. values',
                         'Number of facades', 'Number of rooms', 
                         'Living area',
                         'Furnished','Fully equipped kitchen','Kitchen values',
                         'Surface of the land',
                         'Primary energy consumption','Energy_classes' ,'CO₂ emission',
                         'Surface of the land', 'Terrace', 'Terrace surface','Garden','Garden surface',
                         'Open fire', 'Swimming pool'])

def clean_and_convert(column):
    column = column.apply(lambda x: re.sub('\D+', '', str(x)))
    column = column.replace('', np.nan).fillna(0).astype(int)
    return column

df['Locality'] = df['Locality'].apply(urllib.parse.unquote)

df['Living area'] = clean_and_convert(df['Living area'])
df['Terrace'] = clean_and_convert(df['Terrace'])
df['Garden'] = clean_and_convert(df['Garden'])
df['Surface of the land'] = clean_and_convert(df['Surface of the land'])

df.to_csv('properties_clean.csv', index=False)
'''

'df = pd.read_csv(\'properties_data.csv\')\ndf = df.drop_duplicates(subset=\'id\')\ndf = df.drop(df[df["Type"].isin(["house group", "apartment group"])].index)\n\nduplicate_columns = df.columns[df.columns.duplicated()]\ndf = df.loc[:, ~df.columns.duplicated()]\n\nfiltered_columns = ["id","location","Zip","Type","Subtype",\n    "Price","Transaction Type","Bedrooms","Living area","Kitchen type",\n    "Furnished","How many fireplaces?","Terrace","Terrace surface",\n    "Garden","Garden surface","Surface of the plot","Number of frontages",\n    "Swimming pool","Building condition","Primary energy consumption"]\ndf = df[filtered_columns]\n\ndf = df.rename(columns={\n    \'location\' :\'Locality\',\n    \'Transaction Type\' : \'Type of sale\',\n    \'Type\' :\'Type of property\',\n    \'Subtype\' : \'Subtype of property\',\n    \'Number of frontages\': \'Number of facades\',\n    \'Bedrooms\':\'Number of rooms\',\n    \'Surface of the plot\' :\'Surface of the land\',\n    \'Kitchen type\' : 

In [15]:
# df.describe()

In [16]:
df_houses = df[df['Type of property'] == 'house']
df_apartments = df[df['Type of property'] == 'apartment']

all_missing_value_df = df.isnull().mean() * 100
houses_missing_value_df = df_houses.isnull().mean() * 100
apartments_missing_value_df = df_apartments.isnull().mean() * 100

print("all_missing_value")
print(all_missing_value_df)
print("")
print("houses_missing_value")
print(houses_missing_value_df)
print("")
print("apartments_missing")
print(apartments_missing_value_df)

all_missing_value
Locality                       0.000000
Zip                            0.000000
Type of property               0.000000
Subtype of property            0.000000
Type of sale                   0.000000
Price                          0.000000
Building condition            15.768164
Building Cond. values          0.000000
Number of facades             23.305602
Number of rooms                1.835829
Living area                    0.000000
Furnished                      0.000000
Fully equipped kitchen        21.813644
Kitchen values                 0.000000
Surface of the land           49.833611
Primary energy consumption     0.000000
Energy_classes                 0.000000
Terrace                        0.000000
Terrace surface               23.893511
Garden                         0.000000
Garden surface                12.246256
Open fire                      0.000000
Swimming pool                  0.000000
dtype: float64

houses_missing_value
Locality                 

In [17]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})

missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df.describe()

,percent_missing
count,23.000000
mean,6.465070
std,12.638176
min,0.000000
25%,0.000000
50%,0.000000
75%,7.041043
max,49.833611


In [18]:
most_exp_zip = df.groupby('Zip')['Price'].mean().idxmax()
less_exp_zip = df.groupby('Zip')['Price'].mean().idxmin()
print(most_exp_zip)
print(less_exp_zip)



8902
5680


In [19]:

def get_province(zip_code):
    if 1000 <= zip_code <= 1299:
        return 'Brussels Capital Region'
    elif 1300 <= zip_code <= 1499:
        return 'Walloon Brabant'
    elif 1500 <= zip_code <= 1999 or 3000 <= zip_code <= 3499:
        return 'Flemish Brabant'
    elif 2000 <= zip_code <= 2999:
        return 'Antwerp'
    elif 3500 <= zip_code <= 3999:
        return 'Limburg'
    elif 4000 <= zip_code <= 4999:
        return 'Liège'
    elif 5000 <= zip_code <= 5999:
        return 'Namur'
    elif 6000 <= zip_code <= 6599 or 7000 <= zip_code <= 7999:
        return 'Hainaut'
    elif 6600 <= zip_code <= 6999:
        return 'Luxembourg'
    elif 8000 <= zip_code <= 8999:
        return 'West Flanders'
    elif 9000 <= zip_code <= 9999:
        return 'East Flanders'
    else:
        return 'Unknown'
        
df['Province'] = df['Zip'].apply(get_province)


df = df.reindex(columns=['Locality',"Zip",'Province', 
                         'Type of property', 'Subtype of property',
                         'Type of sale', 'Price',
                         'Building condition','Building Cond. values',
                         'Number of facades', 'Number of rooms', 
                         'Living area',
                         'Furnished','Fully equipped kitchen','Kitchen values',
                         'Surface of the land',
                         'Primary energy consumption','Energy_classes' ,
                         'Terrace', 'Terrace surface','Garden','Garden surface',
                         'Open fire', 'Swimming pool'])

df.to_csv("clean_data.csv")